In [ ]:
'''Notes from Kevin:
This is javascript!

Style stuff
Google API log-in call (script source- loads library?)
Create some variables, assign a couple locations
Function initialize:
    Where to center map, what zoom level to start at
    Map options
    Create new map (google.maps.Map)
    
    block_geo_dict (json object), safe helps get data in
    then addGeoJson to add in layers
    
    How to pull in data?
        make query, go in line by line
        See Zein/Kevin's example
    
    Slider:
'''

In [ ]:
'''
Mapping desires:

Google API

Map data by census group block and census track
Look at clusters for each and pick final model

Slider for year

Hover-over with data distribution for neighborhood
'''

In [45]:
import psycopg2
import pandas as pd
import json

# Connect to database.

conn_dict = {'dbname':'oakland', 'user':'danaezoule', 'host':'/tmp'}
conn = psycopg2.connect(dbname=conn_dict['dbname'], user=conn_dict['user'], host=conn_dict['host'])
c = conn.cursor()

#add a step here to speed things up:
#check if file exists, if not, repull and pickle it
#see kevin's code: safe_walk_app/safe_walk_app.py save_geo_dict

def set_query():
	'''
	Input:
	- None

	Output:
	- Query string
	'''

	# Convert postGIS polygon to GeoJSON geometry.
	# Join on area_features to ensure that only areas with crime counts are included

	map_query = '''
				SELECT ST_AsGeoJSON(shp_table.wkb_geometry) as geom
				FROM shp_table
				JOIN area_features
				ON shp_table.ogc_fid = area_features.ogc_fid
				;'''

	return map_query

def query_db(conn, query):
	'''
	Input:
	- Connection, Query

	Output:
	- Data
	'''
	cur = conn.cursor()
	cur.execute(query)
	df = pd.DataFrame(cur.fetchall())
	df.columns = ['geom']
	return df

def to_json(row):
	'''
	Input:
	- Row of dataframe

	Output:
	- GeoJSON file
	'''

	geo_json = {"type":"Feature", "geometry":json.loads(row['geom'])}
	return geo_json

def join_json():
	'''
	Input:

	Output:
	- GeoJSON Feature Collection (ready for mapping)
	'''

	df = query_db(conn, set_query())
	geo_list = []
	for _, row in df.iterrows():
		geo_list.append(to_json(row))
		
	return json.dumps( { "type":"FeatureCollection", 
				"features":geo_list } )

In [46]:
df = query_db(conn, set_query())

In [47]:
join_json()

'{"type": "FeatureCollection", "features": [{"geometry": {"type": "MultiPolygon", "coordinates": [[[[-122.132378, 37.717838], [-122.132367, 37.717869], [-122.13234, 37.717912], [-122.132304, 37.717957], [-122.132254, 37.717997], [-122.132199, 37.718033], [-122.132026, 37.717666], [-122.131981, 37.71754], [-122.131952, 37.717401], [-122.131944, 37.717357], [-122.131926, 37.7173], [-122.131464, 37.716477], [-122.130998, 37.715597], [-122.130979, 37.715561], [-122.130933, 37.715484], [-122.130871, 37.715411], [-122.13081, 37.715363], [-122.130742, 37.715327], [-122.130669, 37.715306], [-122.1306, 37.715299], [-122.130524, 37.715301], [-122.130446, 37.715309], [-122.130415, 37.715314], [-122.130321, 37.715331], [-122.130291, 37.715337], [-122.130213, 37.715341], [-122.130106, 37.715338], [-122.130021, 37.71533], [-122.129841, 37.715315], [-122.129731, 37.715306], [-122.129217, 37.715248], [-122.12895, 37.715219], [-122.128801, 37.715202], [-122.127051, 37.715694], [-122.124352, 37.716525],